In [1]:
import sqlite3
import pandas as pd
from pandas.core.frame import DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

from matplotlib import pyplot as plt
from time import sleep

import matplotlib


# import gnuplotlib as gp
import numpy as np

import sys

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
import requests

In [4]:
def get_df(f_path: str = "/home/kefei/.volttron/agents/"
                       "ba3caba2-2e44-49a3-a771-eaccf6b7bc3e/sqlhistorianagent-4.0.0/data/platform.historian.sqlite"
           ) -> pd.DataFrame:
    # f_path = "/home/kefei/.volttron/agents/85736715-3354-4b70-94d7-eb2ca6e88e48/sqlhistorianagent-4.0.0/data/platform.historian.sqlite"
    con = sqlite3.connect(f_path)

    # df = pd.read_sql_query("SELECT name FROM sqlite_schema WHERE type ='table' AND name NOT LIKE 'sqlite_%'", con)
    # df.head()

    df_data = pd.read_sql_query(sql="select * from data", con=con)
    # df_data

    df_topics = pd.read_sql_query(sql="select * from topics", con=con)
    # df_topics
    con.commit()  # to make sure refresh

    df_join = pd.merge(df_data, df_topics, on='topic_id')
    # df_join.head()

    return df_join

In [35]:
df = get_df()
df.tail()

,ts,topic_id,value_string,topic_name,metadata
49763,2022-05-18T19:51:30.023592+00:00,88,2,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49764,2022-05-18T19:51:35.020461+00:00,88,1,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49765,2022-05-18T19:51:40.021101+00:00,88,7,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49766,2022-05-18T19:51:45.024189+00:00,88,3,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49767,2022-05-18T19:51:50.022036+00:00,88,2,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"


In [43]:
# inspect here
df = get_df()
df.sort_values('ts', ascending=False).head(8)


,ts,topic_id,value_string,topic_name,metadata
48715,2022-05-18T19:57:12.781411+00:00,83,28193.2651,campus/building/restapi/bitcoin_eur,"{""units"": ""eur"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
48099,2022-05-18T19:57:12.781411+00:00,81,29311.6512,campus/building/restapi/bitcoin_usd,"{""units"": ""usd"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
48407,2022-05-18T19:57:12.781411+00:00,82,23954.1848,campus/building/restapi/bitcoin_gbp,"{""units"": ""gbp"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49023,2022-05-18T19:57:12.781411+00:00,84,false,campus/building/restapi/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
50279,2022-05-18T19:57:10.023190+00:00,88,10,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49651,2022-05-18T19:57:10.023190+00:00,86,0.8660254037844386,campus/building/user-fake/EKG,"{""units"": ""degree"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49337,2022-05-18T19:57:10.023190+00:00,85,false,campus/building/user-fake/Heartbeat,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49965,2022-05-18T19:57:10.023190+00:00,87,-98.84235250156972,campus/building/user-fake/OutsideAirTemperature1,"{""units"": ""F"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"


In [ ]:
topic_prfix = "campus/building/restapi/"
topic_prfix = "campus/building/user-fake/"

# inspect here
df = get_df()
df_filetered = df.sort_values('ts', ascending=False)
df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)

In [26]:
from IPython.display import display, clear_output

In [47]:


topic_prfix = "campus/building/restapi/"
topic_prfix = "campus/building/user-fake/"
topic_prfix = "campus/building/"

# inspect here
df = get_df()
df.ts = pd.to_datetime(df.ts)
df_filetered = df.sort_values('ts', ascending=False)
df_filetered = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)]
df_play = df_filetered.sort_values('ts', ascending=False).head(8)

old_index = df_play.index
count = 500
while count > 0:
    df = get_df()
    df.ts = pd.to_datetime(df.ts)
    df_filetered = df.sort_values('ts', ascending=False)
    df_filetered = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)]
    df_play = df_filetered.sort_values('ts', ascending=False).head(8)
    new_index = df_play.index.copy()
    index_diff = new_index.difference(old_index)
    old_index = new_index
    if len(index_diff) > 0:
#         print(df_filetered.loc[index_diff].head(8))
        print(df_display)
#         display(df_display)
    else:
        print("no new data published. Count at: ", count)
    count -= 1
#     print("count", count)
    sleep(2)
    clear_output(wait=True)

                                     ts  topic_id value_string  \
50152  2022-05-18T19:55:40.022455+00:00        88           10   
50153  2022-05-18T19:55:45.021146+00:00        88            7   
50154  2022-05-18T19:55:50.022184+00:00        88            6   
50155  2022-05-18T19:55:55.021520+00:00        88            5   

                                  topic_name  \
50152  campus/building/user-fake/SampleLong1   
50153  campus/building/user-fake/SampleLong1   
50154  campus/building/user-fake/SampleLong1   
50155  campus/building/user-fake/SampleLong1   

                                                    metadata  
50152  {"units": "1", "type": "integer", "tz": "US/Pacific"}  
50153  {"units": "1", "type": "integer", "tz": "US/Pacific"}  
50154  {"units": "1", "type": "integer", "tz": "US/Pacific"}  
50155  {"units": "1", "type": "integer", "tz": "US/Pacific"}  


KeyboardInterrupt: 

In [11]:
df_hist = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
df_hist_2 = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(7)

In [12]:
df_hist_2.index

Int64Index([48243, 48183, 48303, 48363, 48362, 48242, 48182], dtype='int64')

In [14]:
df_hist.index.difference(df_hist_2.index) # order matters

Int64Index([48302], dtype='int64')

In [15]:
df_hist_2.index.difference(df_hist.index)

Int64Index([], dtype='int64')